# BERT + LoRA 및 기타 테크닉을 이용하여 Classification 구현하기 (Large Model Version)

Author : 정상근 (hugmanskj@gmail.com)

> 알림>
> 교육용 목적으로 제작된 코드입니다.

이 노트북에서는 BERT 모델을 LoRa를 이용하여 메모리를 적게 사용하는 방법을 실습합니다.
활용 데이터는 IMDB이며, **BERT Large** +LoRA를 이용하여 분류문제를 구현해봅니다.

이번 실습에서는 좀 더 큰 BERT를 활용하기 때문에 GPU 메모리가 부족할 수 있습니다. 이런 문제를 해결하기 위해 Quantization 및 다른 테크닉들을 적용하여 큰 모델을 활용하여 Fine-tuning을 진행해보겠습니다.

아래의 소프트웨어와 프레임워크를 사용해서 진행하겠습니다.

(소요시간 : 1시간, colab free 기준)

In [48]:
from IPython.display import Markdown, display

In [49]:
!pip install -q --upgrade datasets fsspec
!pip install evaluate==0.4.5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [50]:
import os

# 첫 번째 GPU만 사용하도록 설정 (본 실습환경 때문)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [51]:
import os
os.environ["WANDB_MODE"] = "disabled"

아래 사이트에서 Dataset을 살펴볼 수 있습니다.

https://huggingface.co/datasets/imdb/viewer/plain_text/train

## 향후 쓰일 여러가지 utility function 준비

In [52]:
from pynvml import *

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable : {100 * trainable_params / all_param}%"
    )
    return trainable_params


def compare_param(ori_p, peft_p):
    """
    Compare two parameter numbers
    """
    print(f"\n# Trainable Parameter \nBefore: {ori_p:>14,d} \nAfter:  {peft_p:>14,d} \nPercentage: {round(peft_p / ori_p * 100, 2)}%")


def show_trainable_structure(model):
    """
    Print only the trainable parameters only with size
    """
    num_totals = 0
    for _, param in model.named_parameters():
        if param.requires_grad:
            print( f"{_} \t {param.shape} \t\t {param.numel():,}" )
            num_totals += 1

    print(f"\nTotal Number of Parameter Names : {num_totals:,}")

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()


## 평가 Metric 준비하기

In [53]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## IMDB Dataset

#### Dataset 준비

In [54]:
## 만약 아래 코드에서 오류가 난다면, colab 기준 커널을 다시 시작하시면 됩니다.

In [55]:
## It takes time! (about 10min)
from datasets import load_dataset
dataset = load_dataset("imdb")

In [56]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

#### Model ID 설정

In [57]:
MODEL_ID = "bert-large-uncased"

#### Tokenization

tokenized_datasets 에 미리 dataset 전체를 tokenization 해 둡니다.

In [58]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

테스트를 위해 작은 데이터 준비

In [59]:
# -- small dataset (about 1hour)
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(4000))
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# -- full dataset (> 15 hours)
#train_dataset = tokenized_datasets["train"]
#test_dataset = tokenized_datasets["test"]

이번 실습에서는, GPU 메모리에서 한번에 훈련하기 버거운 모델을 가져와서 몇가지 테크닉을 통해 훈련을 가능케 하는 테크닉을 살펴보도록 하겠습니다. 더불어 이러한 테크닉들은 LoRA랑도 같이 활용해 보겠습니다.

In [60]:
## Model Loading
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
ori_p = print_trainable_parameters(model)

trainable params: 335,143,938 || all params: 335,143,938 || trainable : 100.0%


## Big Model 을 작은 GPU 에 넣는 기법들

실용적으로 사용할 만한 테크닉이 몇 가지를 소개합니다. 계속해서 새로운 테크닉이 나오고, 사용할 수 있는 툴도 다양해지고 있기 때문에 모든 것을 소개해드리기는 힘듭니다만, Huggingface 상황에서 쉽게 적용할 수 있는 몇가지 테크닉을 소개합니다.




### [모델 로딩시 적용] 8Bit 로 Model 로딩해오기
Pretrained Model 을 로딩해올 때, load_in_8bit=True 옵션은 딥러닝 모델 훈련과 추론 과정에서 메모리 사용량을 크게 줄이는 방법입니다. 이 옵션을 활성화하면 모델의 가중치와 활성화들이 8비트 정밀도로 저장되고 처리됩니다. 이 접근법의 주요 장점과 효과는 다음과 같습니다:

- 메모리 사용량 감소: 8비트 정밀도로 모델을 로드하면 기존 32비트(float) 형식에 비해 메모리 사용량이 대폭 감소합니다. 이는 특히 GPU 메모리가 제한된 상황에서 유용하며, 더 큰 모델을 사용하거나 더 큰 배치 크기로 훈련하는 데 도움이 됩니다.
- 훈련 및 추론 속도 향상: 메모리 사용량이 줄어들면 데이터 전송 속도가 빨라지고, 연산이 더 효율적으로 이루어질 수 있습니다. 결과적으로, 모델 훈련과 추론 속도가 향상될 수 있습니다.


#### 주의사항
- 정밀도와 성능: 8비트 정밀도는 32비트보다 낮기 때문에, 때때로 모델의 성능(정확도 등)에 영향을 줄 수 있습니다. 따라서 성능에 민감한 작업에서는 8비트와 32비트 모델의 성능을 비교하는 것이 중요합니다.
- 하드웨어 호환성: 모든 하드웨어가 8비트 연산을 지원하는 것은 아닙니다. 따라서 8비트 모델을 사용하기 전에 하드웨어의 호환성을 확인하는 것이 좋습니다.

----


### [모델에 적용] Gradient Checkpointing 적용하기
huggingface의 경우 'model.gradient_checkpointing_enable()' 라는 함수를 제공합니다. 이 함수는 딥러닝 모델, 특히 메모리 집약적인 대규모 모델의 훈련 시 메모리 효율성을 크게 향상시키는 기법인 Gradient Checkpointing을 활성화합니다.

표준 backpropagation에서 그래디언트를 계산하기 위해, 신경망은 forward pass 동안의 activations (각 레이어의 출력)을 저장합니다. 이 activations은 backward pass에서 그래디언트를 계산하는 데 사용됩니다. 하지만 많은 레이어를 가진 딥 네트워크에서 이 activations을 모두 저장하는 것은 상당한 양의 메모리를 소비할 수 있습니다.

이 기법의 주요 장점과 효과는 다음과 같습니다:

- 메모리 사용량 감소: Gradient Checkpointing은 전방향 전달(forward pass) 동안의 모든 활성화(activations)를 저장하는 대신, 특정 레이어에서만 활성화를 저장하고, 필요할 때 나머지를 다시 계산합니다. 이는 전체적인 메모리 사용량을 크게 줄여줍니다.

- 대규모 모델 훈련 가능: 이 기법을 사용하면 메모리 사용량이 감소함에 따라, 기존에는 GPU 메모리 제약으로 인해 훈련하기 어려웠던 대규모 모델도 훈련할 수 있게 됩니다.

- 배치 크기 증가: 메모리 절감으로 인해 더 큰 배치 크기로 훈련을 진행할 수 있게 되며, 이는 모델의 성능과 안정성에 긍정적인 영향을 미칠 수 있습니다.


#### 주의사항
- 계산 오버헤드: 활성화를 다시 계산해야 하므로, 훈련 시간이 다소 증가할 수 있습니다. 이는 메모리 절약과 계산 시간 사이의 트레이드오프를 고려해야 합니다.
- 하이퍼파라미터 조정: Gradient Checkpointing을 사용할 때는 학습률 및 다른 하이퍼파라미터를 조정할 필요가 있을 수 있습니다.
- 모델에 따라 적용이 안되는 경우도 있습니다.

### [모델에 적용] LoRA 적용하기

LoRA(Low-Rank Adaptation)는 특히 사전 훈련된 딥러닝 모델, 예를 들어 Transformer 기반 모델의 효율적인 미세 조정을 가능하게 하는 기술입니다. lora_config를 사용하여 LoRA를 적용하면 다음과 같은 효과를 기대할 수 있습니다:

- 효율적인 파라미터 수정: LoRA는 모델의 기존 가중치를 직접 수정하지 않고, 저차원 행렬을 도입하여 기존 가중치에 적용합니다. 이 접근법은 모델의 파라미터 수를 크게 늘리지 않으면서도 사전 훈련된 모델의 성능을 개선할 수 있습니다.

- 빠른 미세 조정 및 적응: LoRA를 적용한 모델은 새로운 데이터 또는 특정 작업에 대해 빠르게 미세 조정될 수 있습니다. 이는 전체 모델을 처음부터 훈련시키는 것보다 시간과 자원을 절약할 수 있게 해줍니다.

- 유연한 훈련과 확장성: lora_config에서는 LoRA의 다양한 매개변수(예: 저차원 행렬의 랭크)를 조정할 수 있어, 모델의 크기와 복잡도를 효과적으로 관리할 수 있습니다. 이를 통해 모델의 용량과 성능 사이에서 균형을 맞출 수 있습니다.

- 기존 가중치 보존: LoRA는 사전 훈련된 모델의 가중치를 보존하면서 새로운 데이터에 대한 학습을 가능하게 합니다. 이는 전이 학습(Transfer Learning)에서 중요한 요소로, 모델이 기존에 학습한 지식을 유지하면서 새로운 정보를 통합할 수 있게 해줍니다.

#### 주의사항
- 하이퍼파라미터 조정: LoRA의 효과는 lora_config에서 설정한 하이퍼파라미터에 크게 의존합니다. 적절한 랭크와 학습률 등을 실험을 통해 찾는 것이 중요합니다.

- 특정 모델 및 작업에 최적화: LoRA는 특히 Transformer 기반 모델에 적합합니다. 따라서 모델의 종류와 특정 작업에 따라 LoRA의 적용 방식과 효과가 달라질 수 있습니다.

----

### [훈련 시 적용] fp16 옵션 켜기
fp16=True 설정은 딥러닝 모델 훈련 시 16비트 부동소수점(FP16, Half Precision)을 사용하도록 지정하는 옵션입니다. 이 설정은 모델 훈련의 메모리 사용량을 줄이고, 훈련 속도를 높이는 데 도움이 됩니다. fp16 모드의 주요 장점과 효과는 다음과 같습니다:

- 메모리 사용량 감소: FP16을 사용하면 32비트 부동소수점(FP32, Full Precision)에 비해 메모리 사용량이 절반으로 줄어듭니다. 이는 특히 GPU 메모리가 제한된 상황에서 유용하며, 더 큰 모델을 사용하거나 더 큰 배치 크기로 훈련하는 데 도움이 됩니다.

- 훈련 및 추론 속도 향상: 메모리 대역폭이 감소하고 연산 속도가 빨라지기 때문에, 전체적인 모델 훈련과 추론 속도가 향상됩니다. FP16은 특히 GPU에서의 행렬 연산에서 더 빠른 속도를 제공합니다.

#### 주의사항
- 정밀도와 성능: FP16은 FP32보다 낮은 정밀도를 가지므로, 때때로 모델의 성능(정확도 등)에 영향을 줄 수 있습니다. 성능 저하를 최소화하기 위해, 많은 딥러닝 프레임워크들은 FP16과 FP32를 혼합하여 사용하는 Mixed Precision 훈련을 지원합니다.

- 하드웨어 호환성: 모든 GPU가 FP16 연산을 최적화하여 지원하는 것은 아닙니다. 따라서 FP16 모드를 사용하기 전에 하드웨어의 호환성을 확인하는 것이 중요합니다.
----


### [훈련 시 적용] Graident Accumulation 사용하기

gradient_accumulation_steps 설정은 딥러닝 모델 훈련 시 배치에 대한 그래디언트를 즉시 업데이트하는 대신, 여러 스텝에 걸쳐 그래디언트를 누적(accumulate)한 후에 업데이트하는 방식을 정의합니다. 이 옵션의 주요 장점과 효과는 다음과 같습니다:

- 큰 배치 크기 효과: 실제 물리적 배치 크기는 제한되어 있지만, gradient_accumulation_steps를 증가시킴으로써 더 큰 배치 크기의 효과를 낼 수 있습니다. 이는 메모리 한계 때문에 큰 배치 크기를 직접 사용할 수 없는 경우에 특히 유용합니다.

- 메모리 효율성 증가: 더 큰 배치 크기를 사용하는 것과 유사한 효과를 내면서도, 각 스텝에서 처리하는 실제 데이터 양은 작게 유지되므로 GPU 메모리 사용량을 줄일 수 있습니다.

- 모델 성능 및 안정성 개선: 더 큰 배치 크기로 인한 이점을 통해 모델의 성능이 향상되고, 훈련 과정이 더 안정적으로 진행될 수 있습니다. 큰 배치 크기는 그래디언트 업데이트의 분산을 줄여주며, 이는 일반적으로 더 안정적인 최적화 과정으로 이어집니다.

#### 주의사항
- 훈련 시간: 그래디언트 누적은 실제 업데이트가 더 드물게 일어나므로, 같은 수의 에포크를 완료하는 데 더 긴 시간이 소요될 수 있습니다.

- 하이퍼파라미터 조정: gradient_accumulation_steps를 변경하면 학습률과 같은 다른 하이퍼파라미터를 재조정할 필요가 있을 수 있습니다.

## Big Model 을 작은 GPU 에 넣기

In [62]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Fri Jul 18 02:15:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100 80GB PCIe          On  |   00000000:00:0C.0 Off |                    0 |
| N/A   45C    P0             77W /  300W |   34006MiB /  81920MiB |     21%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Quantized Model 준비하기

In [63]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

tokenizer_id = MODEL_ID

# 8bit quantization 설정
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

# 모델 로딩
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config,
    device_map='auto',
    trust_remote_code=True,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### LoRA 적용하기

In [64]:
import torch
from transformers import AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model

# LoRA 설정 객체 정의
peft_config = LoraConfig(
    task_type="SEQ_CLS",       # 시퀀스 분류
    inference_mode=False,      # 학습 모드
    r=32,                      # LoRA rank
    lora_alpha=1,              # 스케일링
    lora_dropout=0.1,          # 드롭아웃 비율
)

# 3) PEFT-LoRA 어댑터 생성
model = get_peft_model(model, peft_config)

# 4) 이제 Trainer 등에 넘겨 fine-tuning 가능
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 1024, padding_idx=0)
          (position_embeddings): Embedding(512, 1024)
          (token_type_embeddings): Embedding(2, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-23): 24 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear8bitLt(
                    (base_layer): Linear8bitLt(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
            

In [65]:
ori_p = print_trainable_parameters(model)

trainable params: 3,147,778 || all params: 338,291,716 || trainable : 0.9304921909468218%


### Gradient Checkpointing 적용하기

In [66]:
# Gradient Checkpointing 활성화
model.gradient_checkpointing_enable()

In [67]:
lora_p = print_trainable_parameters(model)

trainable params: 3,147,778 || all params: 338,291,716 || trainable : 0.9304921909468218%


### FP16 및 Gradient Accumuation 적용하기

In [68]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    ## ---- Epoch & Batch ----- ##
    num_train_epochs=20,             # 훈련 에폭 수
    per_device_train_batch_size=50,  # 디바이스 당 훈련 배치 크기
    auto_find_batch_size=True,

    ## ------- PEFT------ ----- ##
    gradient_accumulation_steps=4,   # <-- enable
    #fp16=False,                       # <-- enable (GPU 아키텍쳐마다 다릅니다. 실습을 위해 해제합니다.)
    #bf16=True,
    optim="adamw_bnb_8bit",

    ## ---- Learning Rate ----- ##
    warmup_steps=500,                # 워밍업을 위한 스텝 수
    weight_decay=0.01,               # 가중치 감소율
    learning_rate=1e-4,              # 데이터 셋 마다 적정값을 찾아야 합니다.

    max_grad_norm=1.0,

    ## ---- GPU & NODES ----- ##
    eval_strategy="epoch",

    ## - Logging & Checkpoint - ##
    output_dir='./results',          # 결과와 체크포인트를 저장할 디렉토리

    logging_steps=10,                # 로그를 기록할 스텝 간격
    #save_steps=1000,                 # 체크포인트를 저장할 스텝 간격
)


>Note!

fp16=True 옵션은 stability 이슈가 종종 생길 수 있습니다. 작은 데이터로 돌려보고, 상황을 봐서 적용하시면 됩니다.

### Trainer 설정

In [69]:
trainer = Trainer(
    model=model,
    args=training_args,

    # on Which Dataset
    train_dataset=train_dataset, # < train
    eval_dataset=test_dataset,   # < test

    # computing metrics (during the training)
    compute_metrics=compute_metrics,
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [70]:
import warnings

# huggingface 훈련 중 불필요한 Warning 이 많이 발생함 (epoch 별로)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
trainer.train()

가지고 있는 메모리 용량등을 확인 한 후 batch size 나 gradient accumulation step등을 조정하시면서 성능을 최적화 하면 됩니다.